<a href="https://colab.research.google.com/github/rohankj01/My-work-Deep-into-CNNs/blob/main/W2-3/Competitions/Kaggle_Hackathon1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle competitions download -c tabular-playground-series-jun-2021

In [ ]:
! mkdir train
! unzip train.csv.zip -d train
! mkdir test
! unzip test.csv.zip -d test

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [ ]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('test/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train = train.drop(['Name','Ticket','PassengerId'], axis=1)

In [ ]:
one_hot_train1 = pd.concat([train, pd.get_dummies(train[['Embarked','Sex']])], axis=1)
one_hot_train2 = pd.concat([one_hot_train1, pd.get_dummies(train['Pclass'])], axis=1)
one_hot_train2['FamNo'] = one_hot_train2['SibSp'] + one_hot_train2['Parch']
one_hot_train2['Age'] = one_hot_train2['Age'].replace(np.nan, 39.00)
one_hot_train2['Age'] = one_hot_train2['Age']/87.00

one_hot_train2['Fare'] = one_hot_train2['Fare']/(one_hot_train2['Fare'].max())
one_hot_train2['FamNo'] = one_hot_train2['FamNo']/(one_hot_train2['FamNo'].max())
processed_data = one_hot_train2.drop(['Sex','Pclass','Embarked','Embarked_C','Sex_female','SibSp','Cabin','Parch',1], axis=1)

In [ ]:
processed_data.head()

In [ ]:
test = test.drop(['Name','Ticket','PassengerId'], axis=1)

In [ ]:
one_hot_test1 = pd.concat([test, pd.get_dummies(test[['Embarked','Sex']])], axis=1)
one_hot_test2 = pd.concat([one_hot_test1, pd.get_dummies(test['Pclass'])], axis=1)
one_hot_test2['FamNo'] = one_hot_test2['SibSp'] + one_hot_test2['Parch']
one_hot_test2['Age'] = one_hot_test2['Age'].replace(np.nan, 39.00)
one_hot_test2['Age'] = one_hot_test2['Age']/87.00

one_hot_test2['Fare'] = one_hot_test2['Fare']/(one_hot_test2['Fare'].max())
one_hot_test2['FamNo'] = one_hot_test2['FamNo']/(one_hot_test2['FamNo'].max())
processed_data_test = one_hot_test2.drop(['Sex','Pclass','Embarked','Embarked_C','Sex_female','SibSp','Cabin','Parch',1], axis=1)

In [ ]:
processed_data_test

In [ ]:
features, targets = processed_data.drop(['Survived'], axis=1), processed_data['Survived']

In [ ]:
# Activation (sigmoid) function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))
def error_formula(y, output):
    return - y*np.log(output) - (1 - y) * np.log(1-output)

In [ ]:
def error_term_formula(x, y, output):
    error_term = (y-output)*sigmoid_prime(x)*x
    return error_term

In [ ]:
# Neural Network hyperparameters
epochs = 100
learnrate = 0.001

# Training function
def train_nn(features, targets, epochs, learnrate):
    
    # Use to same seed to make debugging easier
    np.random.seed(42)

    n_records, n_features = features.shape
    last_loss = None

    # Initialize weights
    weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x, y in zip(features.values, targets):
            # Loop through all records, x is the input, y is the target

            # Activation of the output unit
            #   Notice we multiply the inputs and the weights here 
            #   rather than storing h as a separate variable 
            output = sigmoid(np.dot(x, weights))

            # The error term
            error_term = error_term_formula(x, y, output)

            # The gradient descent step, the error times the gradient times the inputs
            del_w += error_term

        # Update the weights here. The learning rate times the 
        # change in weights
        # don't have to divide by n_records since it is compensated by the learning rate
        weights += learnrate * del_w #/ n_records  

        # Printing out the mean square error on the training set
        if e % (epochs / 10) == 0:
            out = sigmoid(np.dot(features, weights))
            loss = np.mean(error_formula(targets, out))
            print("Epoch:", e)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            print("=========")
    print("Finished training!")
    return weights
    
weights = train_nn(features, targets, epochs, learnrate)